In [40]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
reload(Molecules)
reload(D)
import sys,os
sys.path.insert(0,'../')
import StatPol as SP
reload(SP)

<module 'StatPol' from '../StatPol.py'>

In [41]:
def get_converged_input_energy(dataset,rtol,atol):
    data=dataset.seek_convergence(atol=atol,rtol=rtol,attribute='energy')
    return dataset.runs[dataset.names.index(D.name_from_id(data[0]))]['input'],data[1]

def find_gs_domain(label,rtol,atol,path,inp_rmult,posinp,code):
    crmult=inp_rmult['dft']['rmult'][0]
    rmult_fine=inp_rmult['dft']['rmult'][1]
    rmult_list=map(float,range(int(crmult),11))
    seek_for_rmult = D.Dataset(label=label+'(GS)',run_dir=path,posinp=posinp)
    for rm in rmult_list:
        inp_rmult.set_rmult(coarse=rm,fine=rmult_fine)
        seek_for_rmult.append_run(id={'rmult':rm},runner=code,input=inp_rmult)
    input_gs,log_gs=get_converged_input_energy(seek_for_rmult,rtol,atol)
    return {'input_gs':input_gs, 'dataset_gs': seek_for_rmult,'log_gs':log_gs}

def extract_statical_polarizability(label,rtol,atol,path,input_for_alpha,posinp,code,ref):
    ints=[1e-2,5e-3,1e-3,5e-4,1e-4]
    get_alpha=D.Dataset(label=label+'(field)',run_dir=path)
    for f in ints:
        get_alpha.append_run(id={'f':f},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=f))
    try:
        data=get_alpha.seek_convergence(rtol=rtol,atol=atol)
        intensity=data[0]['f']
    except:
        print ('Convergence in field not reached')
        intensity=ints[-1]
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),11))
    final_alpha=D.Dataset(label=label+'(domain)',run_dir=path)
    for rm in rmult_list:
        input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
        final_alpha.append_run(id={'f':intensity,'rmult':rm},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=intensity))
    try:
        import numpy as np
        data_final=final_alpha.seek_convergence(rtol=rtol,atol=atol)
        AuToA = 0.5291772085**3
        alpha_ref_ii = np.array(ref)/AuToA
        eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
        print ('Relative difference in %',eps.tolist())
    except LookupError as e:
        print ('Convergence in domain not reached',e)
        data_final=None
    return {'alpha_convergence': data_final, 'dataset_field': get_alpha, 'dataset_rmult': final_alpha}

In [42]:
def change_workdir(molecule,study,options):
    from futile.Utils import ensure_dir
    datadir=options.get('data_directory','Data')
    for d in [datadir,molecule]:
        ensure_dir(d)
        os.chdir(d)
    path=study[0]+'-'+study[1]
    ensure_dir(path)
    return '-'.join([molecule,path]),path

def gs_study(molecule,study,code,options):
    import Routines as R
    from BigDFT import InputActions as A
    reload(R)
    hgrids=options.get('hgrids')
    rmult_fine=options.get('rmult_fine')
    wf_convergence=options.get('wf_convergence')
    crmult_start=options.get('crmult',4.0)
    rtol=options.get('rtol_gs',10*wf_convergence)
    atol=options.get('atol_gs',1.e-3)
    reference_results=options.get('reference_data')

    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,options)

    posinp=Molecules.Molecule(molecule)
    
    inp = I.Inputfile()
    inp.set_hgrid(hgrids)
    inp.set_rmult(coarse=crmult_start,fine=rmult_fine)
    R.set_xc[study](inp,molecule)
    inp.set_wavefunction_convergence(wf_convergence)
    mpol=int(reference_results[molecule]['mpol_ref'])-1   
    if mpol >0: inp.spin_polarize(mpol)
    data= find_gs_domain(label,10*wf_convergence,atol,path,inp,posinp,code)
    os.chdir(initial_dir)
    
    data['molecule']=molecule
    data['posinp']=posinp
    data['study']=study
    return data 

def alpha_study(gs_data,code,options):
    import copy
    xc_conversion={'lda_pw': 'lda-SPW92', 'pbe': 'pbe', 'pbe0': 'pbe0'}
    rtol=options.get('rtol',1.e-2)
    atol=options.get('atol',1.e-3)
    reference_results=options.get('reference_data')
    study=gs_data['study']
    molecule=gs_data['molecule']
    posinp=gs_data['posinp']
    xc=study[0]
    ref_data=reference_results[molecule][xc_conversion[xc]]
    
    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,options)
    input_gs=copy.deepcopy(gs_data['input_gs'])
        
    data=extract_statical_polarizability(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data)
    os.chdir(initial_dir)
    
    return data

In [43]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))

In [44]:
def get_psp(molecule):
    import Routines as R
    from os import path as p
    possible_psp=['nlcc_aw','nlcc_ss']
    to_do=[True,True]
    for at in R.get_atoms(molecule):
        psp_available = [p.isfile(p.join('psp_'+psp,'psppar.'+at)) for psp in possible_psp]
        to_do = [ a and b for a,b in zip(to_do,psp_available)]
    return ['hgh_k'] + [psp for yes,psp in zip(to_do,possible_psp) if yes]

In [45]:
initial_dir=os.path.abspath(os.path.dirname('/home/genovese/work/execute/StaticalPolarizability/Calculations/'))
os.chdir(initial_dir)

In [46]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True,verbose=False)
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-2,'atol':1.e-3,'reference_data':HG_data}

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/genovese/binaries/1.8.4-genovese/install/bin/bigdft


In [26]:
full_data={}
for molecule in HG_data:
    data=HG_data[molecule]
    if data['spin_pol'] != 'nsp': continue
    for xc in ['lda_pw','pbe','pbe0']:
        psp = ['hgh_k'] if xc != 'pbe' else get_psp(molecule)
        for p in psp:
            print molecule,xc,p
            gs_data=gs_study(molecule,(xc,p),code,options)
            full_data[(molecule,xc,p)]=alpha_study(gs_data,code,options)
            full_data[(molecule,xc,p)].update(gs_data)
#molecule='CO'
#study = ('pbe', 'nlcc_aw')
#results=alpha_study(molecule,study,code,options)

BF lda_pw hgh_k
Fetching results for id " {'rmult': 5.0} "
Convergence reached in Dataset "BF-lda_pw-hgh_k(GS)" for id " {'rmult': 4.0}
Fetching results for id " {'f': 0.005} "
Convergence reached in Dataset "BF-lda_pw-hgh_k(field)" for id " {'f': 0.01}
Fetching results for id " {'rmult': 5.0, 'f': 0.01} "
Fetching results for id " {'rmult': 6.0, 'f': 0.01} "
Fetching results for id " {'rmult': 7.0, 'f': 0.01} "
Convergence reached in Dataset "BF-lda_pw-hgh_k(domain)" for id " {'rmult': 6.0, 'f': 0.01}
('Relative difference in %', [0.5677769587487399, 0.5677769587487399, 0.36667789456298516])
BF pbe hgh_k
Fetching results for id " {'rmult': 5.0} "
Convergence reached in Dataset "BF-pbe-hgh_k(GS)" for id " {'rmult': 4.0}
Fetching results for id " {'f': 0.005} "
Convergence reached in Dataset "BF-pbe-hgh_k(field)" for id " {'f': 0.01}
Fetching results for id " {'rmult': 5.0, 'f': 0.01} "
Fetching results for id " {'rmult': 6.0, 'f': 0.01} "
Fetching results for id " {'rmult': 7.0, 'f': 0

AttributeError: Logfile instance has no attribute 'dipole'

In [47]:
dd=gs_study('O2',('pbe','nlcc_ss'),code,options)
tt=alpha_study(dd,code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "O2-pbe-nlcc_ss(GS)" for id " {'rmult': 5.0} "
Fetching results for id " {'f': 0.01} "
Fetching results for id " {'f': 0.005} "
Convergence reached in Dataset "O2-pbe-nlcc_ss(field)" for id " {'f': 0.01} "
Fetching results for id " {'rmult': 5.0, 'f': 0.01} "
Fetching results for id " {'rmult': 6.0, 'f': 0.01} "
Fetching results for id " {'rmult': 7.0, 'f': 0.01} "
Fetching results for id " {'rmult': 8.0, 'f': 0.01} "
Convergence reached in Dataset "O2-pbe-nlcc_ss(domain)" for id " {'rmult': 7.0, 'f': 0.01} "
('Relative difference in %', [-0.2735657855144162, -0.2735657855144162, 0.23368186274972824])


In [26]:
tt=alpha_study('CO',('lda_pw','hgh_k'),code,{'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-2,'atol':1.e-3, 'crmult':5.0,'reference_data':HG_data})

Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "CO-lda_pw-hgh_k(GS)" for id " {'rmult': 5.0}
Fetching results for id " {'f': 0.005} "
Convergence reached in Dataset "CO-lda_pw-hgh_k(field)" for id " {'f': 0.01}
Fetching results for id " {'rmult': 6.0, 'f': 0.01} "
Fetching results for id " {'rmult': 7.0, 'f': 0.01} "
Convergence reached in Dataset "CO-lda_pw-hgh_k(domain)" for id " {'rmult': 6.0, 'f': 0.01}
('Relative difference in %', [-0.04292247676812145, -0.04292247676812145, -0.3758471694941252])


In [41]:
dd=HG_data['O2']